In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
df = pd.read_csv(url, header=None)
df.columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Print basic information
print(df.head())
print(df.describe())
print(df.info())

# Handling missing and duplicate data
print("Null Values:", df.isnull().sum())
print("Duplicate Values:", df.duplicated().sum())
df = df.drop_duplicates()
df = df.dropna()

# Data normalization
data_normalized = (df - df.mean()) / df.std()

# Define a simple autoencoder model
class SimpleAutoencoder(nn.Module):
    def _init_(self, input_size):
        super(SimpleAutoencoder, self)._init_()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# Initialize model, loss function, and optimizer
input_size = df.shape[1] - 1  # Exclude target variable
model = SimpleAutoencoder(input_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Data preparation
data_normalized_tensor = torch.tensor(data_normalized.drop('Outcome', axis=1).values, dtype=torch.float32)
data_loader = DataLoader(data_normalized_tensor, batch_size=16, shuffle=True)

# Training the model
num_epochs = 10
for epoch in range(num_epochs):
    for data in data_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, data)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')